In [1]:
import pandas as pd
import numpy as np
import glob
import os

def load_rtv_data(rtv_folders):
    """
    加载多个文件夹中的Rtv数据并合并为一个DataFrame。
    
    参数：
    - rtv_folders: 包含Rtv数据的文件夹列表。
    
    返回：
    - 合并后的Rtv数据。
    """
    df_list = []
    for folder in rtv_folders:
        csv_files = glob.glob(os.path.join(folder, '*.csv'))
        for file in csv_files:
            df = pd.read_csv(file, dtype={'Stkcd': str})
            df_list.append(df)
    rtv_data = pd.concat(df_list, ignore_index=True)
    # 转换交易日期为datetime格式
    rtv_data['Trddt'] = pd.to_datetime(rtv_data['Trddt'], format='%Y-%m-%d', errors='coerce')
    # 删除无法解析的日期
    rtv_data = rtv_data.dropna(subset=['Trddt'])
    print(f"合并后的Rtv数据共有 {len(rtv_data)} 条记录。")
    print("Rtv数据样本：")
    print(rtv_data.head())
    return rtv_data

def compute_rtv_indicators(rtv_data):
    """
    计算每只股票每月的Rtv指标（rtv1、rtv3、rtv6、rtv9、rtv12）。
    
    参数：
    - rtv_data: 合并后的Rtv数据。
    
    返回：
    - 包含Rtv指标的DataFrame。
    """
    # 添加 YearMonth 列
    rtv_data['YearMonth'] = rtv_data['Trddt'].dt.to_period('M')
    
    # 按股票和月份汇总每日交易量
    monthly_rtv = rtv_data.groupby(['Stkcd', 'YearMonth']).agg(
        Dnvaltrd_sum=('Dnvaltrd', 'sum'),
        Dnvaltrd_count=('Dnvaltrd', 'count')
    ).reset_index()
    
    print(f"按股票和月份汇总后，数据量：{len(monthly_rtv)} 条记录。")
    print("汇总后的Rtv数据样本：")
    print(monthly_rtv.head())
    
    # 排序
    monthly_rtv = monthly_rtv.sort_values(['Stkcd', 'YearMonth'])
    
    # 计算过去六个月（t-6到t-1）的累计交易量和交易日数
    monthly_rtv['Rolling_sum_Dnvaltrd'] = monthly_rtv.groupby('Stkcd')['Dnvaltrd_sum'].rolling(window=6, min_periods=6).sum().reset_index(0, drop=True)
    monthly_rtv['Rolling_count_Dnvaltrd'] = monthly_rtv.groupby('Stkcd')['Dnvaltrd_count'].rolling(window=6, min_periods=6).sum().reset_index(0, drop=True)
    
    # 计算 Rtv（六个月的平均日交易量）
    monthly_rtv['rtv1'] = monthly_rtv['Rolling_sum_Dnvaltrd'] / monthly_rtv['Rolling_count_Dnvaltrd']
    
    # 过滤掉累计交易日数少于50的记录
    monthly_rtv = monthly_rtv[monthly_rtv['Rolling_count_Dnvaltrd'] >= 50]
    print(f"过滤后，数据量：{len(monthly_rtv)} 条记录。")
    
    # 保留相关列
    monthly_rtv = monthly_rtv[['Stkcd', 'YearMonth', 'rtv1']]
    
    # 计算 rtv3、rtv6、rtv9、rtv12 作为 rtv1 的移动平均值
    holding_periods = [3, 6, 9, 12]
    for period in holding_periods:
        rtv_col = f'rtv{period}'
        monthly_rtv[rtv_col] = monthly_rtv.groupby('Stkcd')['rtv1'].rolling(window=period, min_periods=period).mean().reset_index(0, drop=True)
    
    # 只保留各指标
    rtv_indicators = monthly_rtv[['Stkcd', 'YearMonth', 'rtv1', 'rtv3', 'rtv6', 'rtv9', 'rtv12']]
    
    print("Rtv指标计算完成，样本：")
    print(rtv_indicators.head())
    
    return rtv_indicators

def load_pps_data(pps_file):
    """
    加载Pps数据并处理日期格式。
    
    参数：
    - pps_file: Pps数据文件路径。
    
    返回：
    - 处理后的Pps数据。
    """
    pps_data = pd.read_csv(pps_file, dtype={'Stkcd': str})
    print("Pps数据样本:")
    print(pps_data.head())
    
    # 尝试不同的日期格式，根据实际数据进行调整
    # 这里假设格式为 'YYYY-MM'
    pps_data['Trdmnt'] = pd.to_datetime(pps_data['Trdmnt'], format='%Y-%m', errors='coerce')
    missing_dates = pps_data['Trdmnt'].isna().sum()
    if missing_dates > 0:
        print(f"警告: 有 {missing_dates} 个日期无法转换，可能需要检查数据格式。")
    pps_data = pps_data.dropna(subset=['Trdmnt'])
    pps_data = pps_data.rename(columns={'Trdmnt': 'YearMonth'})
    pps_data['YearMonth'] = pps_data['YearMonth'].dt.to_period('M')
    print("Pps数据转换后样本：")
    print(pps_data.head())
    return pps_data

def compute_pps_indicators(pps_data):
    """
    计算pps1、pps3、pps6指标。
    
    参数：
    - pps_data: 处理后的Pps数据。
    
    返回：
    - 包含pps1、pps3、pps6指标的DataFrame。
    """
    # 按股票和年月排序
    pps_data = pps_data.sort_values(['Stkcd', 'YearMonth'])
    
    # 计算 pps1（t-1月的收盘价）
    pps_data['pps1'] = pps_data.groupby('Stkcd')['Mclsprc'].shift(1)
    
    # 计算 pps3、pps6 作为 pps1 的移动平均值
    pps_data['pps3'] = pps_data.groupby('Stkcd')['pps1'].rolling(window=3, min_periods=3).mean().reset_index(0, drop=True)
    pps_data['pps6'] = pps_data.groupby('Stkcd')['pps1'].rolling(window=6, min_periods=6).mean().reset_index(0, drop=True)
    
    # 保留相关列
    pps_indicators = pps_data[['Stkcd', 'YearMonth', 'pps1', 'pps3', 'pps6']]
    
    print("Pps指标计算完成，样本：")
    print(pps_indicators.head())
    
    return pps_indicators

def save_indicators(rtv_indicators, pps_indicators):
    """
    保存Rtv和Pps指标到独立的CSV文件。
    
    参数：
    - rtv_indicators: 包含Rtv指标的DataFrame。
    - pps_indicators: 包含Pps指标的DataFrame。
    
    返回：
    - 无
    """
    # 保存Rtv指标
    for col in ['rtv1', 'rtv3', 'rtv6', 'rtv9', 'rtv12']:
        output_file = f'{col}.csv'
        df = rtv_indicators[['Stkcd', 'YearMonth', col]].dropna()
        df.to_csv(output_file, index=False)
        print(f"{col} 已成功保存到 {output_file} 文件中。")
    
    # 保存Pps指标
    for col in ['pps1', 'pps3', 'pps6']:
        output_file = f'{col}.csv'
        df = pps_indicators[['Stkcd', 'YearMonth', col]].dropna()
        df.to_csv(output_file, index=False)
        print(f"{col} 已成功保存到 {output_file} 文件中。")

def main():
    # 定义文件路径
    rtv_folders = ['rtv_data_2007', 'rtv_data_2012', 'rtv_data_2017', 'rtv_data_2020', 'rtv_data_2022']
    pps_file = os.path.join('pps_data', 'TRD_Mnth.csv')
    
    # 检查文件夹是否存在
    for folder in rtv_folders:
        if not os.path.exists(folder):
            print(f"错误: 文件夹 {folder} 不存在。请检查路径。")
            return
    if not os.path.exists(pps_file):
        print(f"错误: Pps数据文件 {pps_file} 不存在。请检查路径。")
        return
    
    # 加载Rtv数据
    print("加载人民币交易量（Rtv）数据...")
    rtv_data = load_rtv_data(rtv_folders)
    print(f"Rtv数据加载完成，共 {len(rtv_data)} 条记录。")
    
    # 计算Rtv指标
    print("计算Rtv指标...")
    rtv_indicators = compute_rtv_indicators(rtv_data)
    print(f"Rtv指标计算完成，共 {len(rtv_indicators)} 条记录。")
    
    # 加载Pps数据
    print("加载股价（Pps）数据...")
    pps_data = load_pps_data(pps_file)
    print(f"Pps数据加载完成，共 {len(pps_data)} 条记录。")
    
    # 计算Pps指标
    print("计算Pps指标...")
    pps_indicators = compute_pps_indicators(pps_data)
    print(f"Pps指标计算完成，共 {len(pps_indicators)} 条记录。")
    
    # 保存Rtv和Pps指标到独立的CSV文件
    print("保存Rtv和Pps指标到独立的CSV文件...")
    save_indicators(rtv_indicators, pps_indicators)
    
    print("所有指标已成功计算并保存。")

if __name__ == "__main__":
    main()


错误: 文件夹 rtv_data_2007 不存在。请检查路径。


In [3]:
%%time
import pandas as pd
import numpy as np
import os

def load_and_process_data():
    rtv = pd.read_csv('./indicators/rtv1.csv')
    pps = pd.read_csv('./indicators/pps1.csv')
    price = pd.read_pickle('./price_month.pkl')
    market = pd.read_excel('./market_category.xlsx')
    
    # Convert dates
    price['YearMonth'] = pd.to_datetime(price['date']).dt.strftime('%Y-%m')
    price['risk_free_month'] = price['risk_free_month']*0.01
    
    # Convert stock codes to string
    rtv['Stkcd'] = rtv['Stkcd'].astype(str)
    pps['Stkcd'] = pps['Stkcd'].astype(str)
    price['code'] = price['code'].astype(str)
    market['stock_code'] = market['证券代码'].astype(str).str.slice(0, 6)
    
    mainboard_stocks = market['stock_code'].unique()
    return rtv, pps, price, mainboard_stocks

def calculate_quantiles(data, factor_name, mainboard_stocks, num_groups=5):
    mainboard_mask = data['code'].isin(mainboard_stocks)
    mainboard_data = data[mainboard_mask].copy()
    
    percentile_values = []
    for dt in mainboard_data['YearMonth'].unique():
        monthly_data = mainboard_data[mainboard_data['YearMonth'] == dt]
        if not monthly_data[factor_name].empty:
            perc_values = np.percentile(monthly_data[factor_name].dropna(), [20, 40, 60, 80])
            percentile_values.append({
                'YearMonth': dt,
                'p20': perc_values[0],
                'p40': perc_values[1],
                'p60': perc_values[2],
                'p80': perc_values[3]
            })
    
    percentile_df = pd.DataFrame(percentile_values)
    data['quantile'] = 0
    
    for dt in data['YearMonth'].unique():
        mask = data['YearMonth'] == dt
        dt_percentiles = percentile_df[percentile_df['YearMonth'] == dt]
        if not dt_percentiles.empty:
            bounds = dt_percentiles.iloc[0]
            values = data.loc[mask, factor_name]
            
            data.loc[mask & (values <= bounds['p20']), 'quantile'] = 1
            data.loc[mask & (values > bounds['p20']) & (values <= bounds['p40']), 'quantile'] = 2
            data.loc[mask & (values > bounds['p40']) & (values <= bounds['p60']), 'quantile'] = 3
            data.loc[mask & (values > bounds['p60']) & (values <= bounds['p80']), 'quantile'] = 4
            data.loc[mask & (values > bounds['p80']), 'quantile'] = 5

    return data






def calculate_portfolio_returns(data, holding_periods=[1,3,6,9,12]):
    results = {}
    
    for period in holding_periods:
        shifted_data = data.copy()
        print(data)
        
        if i!=1:
            for i in range(1,period):
                shifted_data['quantile'+str(i)] = shifted_data.groupby('code').shift(i)
            #shifted_data['future_return'] = shifted_data.groupby('code')['return'].shift(-period)
            #shifted_data['future_rf'] = shifted_data.groupby('code')['risk_free_month'].shift(-period)
        
        for i in range(period):
            if i==0:
                grouped = shifted_data.groupby(['YearMonth', 'quantile'])
        
                equal_weighted = grouped['future_return'].mean()
        
                def weighted_return(group):
                    weights = group['market_value'] / group['market_value'].sum()
                    return (group['future_return'] * weights).sum()
            
                value_weighted = grouped.apply(weighted_return)
            ################################################################
        
        for prefix, returns in [('eq_', equal_weighted.unstack()), ('vw_', value_weighted.unstack())]:
            if not returns.empty:
                long_ret = returns[1]
                short_ret = returns[5]
                long_short = long_ret - short_ret
                rf_rate = shifted_data.groupby('YearMonth')['risk_free_month'].first()
                
                results[f'{prefix}period_{period}'] = pd.DataFrame({
                    'long_returns': long_ret,
                    'short_returns': short_ret,
                    'long_short_returns': long_short,
                    'long_alpha': long_ret - rf_rate,
                    'short_alpha': short_ret - rf_rate,
                    'long_short_alpha': long_short - rf_rate
                })
    
    return results

def main():
    rtv, pps, price, mainboard_stocks = load_and_process_data()
    
    for factor_data, factor_name in [(rtv, 'rtv1'), (pps, 'pps1')]:
        print(f"\nProcessing {factor_name}...")
        # Create YearMonth in factor_data
        factor_data['YearMonth'] = pd.to_datetime(factor_data['YearMonth']).dt.strftime('%Y-%m')
        
        factor_data = pd.merge(
            factor_data,
            price[['code', 'YearMonth', 'return', 'market_value', 'risk_free_month']],
            left_on=['Stkcd', 'YearMonth'],
            right_on=['code', 'YearMonth'],
            how='left'
        )
        
        data_with_quantiles = calculate_quantiles(factor_data, factor_name, mainboard_stocks)
        print(data_with_quantiles)
        
        
        returns_dict = calculate_portfolio_returns(data_with_quantiles)
        
        os.makedirs('./portfolio_results', exist_ok=True)
        for period_key, returns_df in returns_dict.items():
            output_file = f'./portfolio_results/{factor_name}_{period_key}.csv'
            returns_df.to_csv(output_file)

if __name__ == "__main__":
    main()


Processing rtv1...
         Stkcd YearMonth          rtv1    code    return  market_value  \
0            1   2007-06  8.846783e+08     NaN       NaN           NaN   
1            1   2007-07  9.057878e+08     NaN       NaN           NaN   
2            1   2007-08  1.000175e+09     NaN       NaN           NaN   
3            1   2007-09  1.051426e+09     NaN       NaN           NaN   
4            1   2007-10  1.080491e+09     NaN       NaN           NaN   
...        ...       ...           ...     ...       ...           ...   
590721  873576   2023-12  3.813677e+07  873576  0.299901    784491.875   
590722  873593   2023-09  1.530891e+08  873593 -0.534710   1295654.625   
590723  873593   2023-10  1.416422e+08  873593 -0.275929   1059003.875   
590724  873593   2023-11  1.442483e+08  873593  0.880506   1991462.750   
590725  873593   2023-12  1.254439e+08  873593 -0.217923   1557478.000   

        risk_free_month  quantile  
0                   NaN         5  
1                  

UnboundLocalError: cannot access local variable 'i' where it is not associated with a value

# 